## Pressure Vessel Optimisation ##

There are 4 variables that describe a pressure vessel:
z1 - Shell thickness
z2 - Head thickness
x3 - Inner radius
x4 - vessel length (excluding head)

Our input variables:
 x1,x2 = 0,0

Decision Vector:
np.array(x1,x2,x3,x4)

Use static penalty



The shell head and thickness can be expressed as
z1 = 0.0625 * x1
z2 = 0.0625 * x2

# Constraints:
x1, x2 are integers between 1 and 99 
x3, x4 are continuous values between 10 and 200

# Best known feasible solution of f(x*) = 5.8853327736×10^3

In [37]:
import numpy as np
import math

np.random.seed(372)




In [38]:
# Initialise counters
f_call_count = 0
g1_call_count = 0
g2_call_count = 0
g3_call_count = 0
g4_call_count = 0

In [39]:
def calculate_zs(xs):
    z1 = (xs[0]*0.0625)
    z2 = (xs[1]*0.0625)
    return z1, z2


In [40]:
def f(xs):
    global f_call_count
    f_call_count += 1

    z1, z2 = calculate_zs(xs)

    x3 = xs[2]
    x4 = xs[3]

    xnew = (1.7781)*(z2)*(x3**2) + (0.6224)*(z1)*(x3)*(x4) + (3.1661)*(z1**2)*(x4) + (19.84)*(z1**2)*(x3)
    return xnew


In [51]:
def check_constraints(xs):
    z1, z2 = calculate_zs(xs)

    if g1(xs) <= z2:
        print("G1 Violation")
        return False
    if g2(xs) <= z1:
        print("G2 Violation")
        return False
    if g3(xs) <= 240:
        print("G3 Violation")
        return False
    if g4(xs) <= -12960000:
        print("G4 Violation")
        return False
    else:
        return True

In [47]:
def g1(xs):
    global g1_call_count
    g1_call_count += 1
    return xs[2]*0.00954
    

def g2(xs):
    global g2_call_count
    g2_call_count += 1
    return xs[2]*0.0193
    
        
def g3(xs):
    global g3_call_count
    g3_call_count += 1
    return xs[3]
  

def g4(xs):
    global g4_call_count
    g4_call_count += 1
    return (-1)*(np.pi)*(xs[2]**2)*(xs[3]) - ((4/3)*np.pi)*(xs[2]**3)


In [52]:
def main():
    xs = np.array([1, 35, 11.678, 88.76])
    z1,z2 = calculate_zs(xs)
    print("Objective function output: ", f(xs))
    print("Constraint function 1 output: ", g1(xs))
    print("Constraint function 2 output: ", g2(xs))
    print("Constraint function 3 output: ", g3(xs))
    print("Constraint function 4 output: ", g4(xs))

    check_constraints(xs)


In [31]:
# RANDOM SEARCH
# [x1,x2](ints 1 <= x <= 99) [x3,x4] (floats 10 <= x <= 200)
def random_search(): 
    K = 4000
    np.random.seed(372)

    x1_best = np.random.randint(low=1, high=99)
    x2_best = np.random.randint(low=1, high=99)

    x3_best = np.random.uniform(low=10, high=200)
    x4_best = np.random.uniform(low=10, high=200)

    best_xs = np.array([x1_best, x2_best, x3_best, x4_best])
    best_output = f(best_xs)
    this_output = 0

    results = np.array([])



    for k in range(K):
        this_x1 = np.random.randint(low=1, high=99)
        this_x2 = np.random.randint(low=1, high=99)

        this_x3 = np.random.uniform(low=10, high=200)
        this_x4 = np.random.uniform(low=10, high=200)

        this_xs = np.array([this_x1, this_x2, this_x3, this_x4])

        this_output = f(this_xs)
        np.append(results,this_output)

        if (abs(5885.3327736 - this_output) < abs(5885.3327736 - best_output)):
            x1_best = this_x1
            x2_best = this_x2
            x3_best = this_x3
            x4_best = this_x4
            best_output = this_output

            print(f'New best! f(x) = {np.format_float_scientific(this_output)}')


        



    # print(x1_best,x2_best,x3_best ,x4_best)

In [35]:
# Simulated Annealing
# Use numpy / scipy to generate gaussian distribution

def simulated_annealing():


    # Hyper-parameters:
    ti = 10
    K = 4000

    x12_bound = (99-1)*0.1
    x34_bound = (200-10)*0.1
    

    x1_best = np.random.randint(low=1, high=99)
    x2_best = np.random.randint(low=1, high=99)
    x3_best = np.random.uniform(low=10, high=200)
    x4_best = np.random.uniform(low=10, high=200)
    best_xs = np.array([x1_best, x2_best, x3_best, x4_best])

    best_xs = np.array([x1_best, x2_best, x3_best, x4_best])


    best_output = f(best_xs)
    y_best = best_output
    x1c = x1_best
    x2c = x2_best
    x3c = x3_best
    x4c = x4_best
    xcs = np.array([x1c,x2c,x3c,x4c])
    yc = y_best
    
    x1p = np.random.normal(loc=x1c, scale=x12_bound)
    
    
    for k in range(K):

        tk = (ti/(k+1))
        
        x1p = np.random.normal(loc=x1c, scale=x12_bound)
        x2p = np.random.normal(loc=x2c, scale=x12_bound)
        x3p = np.random.normal(loc=x3c, scale=x34_bound)
        x4p = np.random.normal(loc=x4c, scale=x34_bound)
        
        xps = np.array([x1p,x2p,x3p,x4p])

        yp = f(xps)

        delta_y = (yp-yc)
        acc_prob = (delta_y*-1)/tk

        if (delta_y <= 0) or (np.random.uniform(low=0.0, high=1.0) < min([math.exp((delta_y*-1)/tk), 1]) ):
            xcs = xps
            yc = yp
        
        if (yp < y_best):
            best_xs = xps
            y_best = yp
            # print(f"new best xs: {best_xs}")
            print(f'New best! f(x) = {np.format_float_scientific(y_best)}')


    return best_xs, y_best
    


In [53]:
main()
random_search()
simulated_annealing()

Objective function output:  572.77017736475
Constraint function 1 output:  0.11140812000000001
Constraint function 2 output:  0.22538540000000004
Constraint function 3 output:  88.76
Constraint function 4 output:  -44699.10187026799
G1 Violation
